# Testing Script

Test whether the various ML interpretability methods within MintPy are functioning correctly. 

In [2]:
import sys, os 
current_dir = os.getcwd()
path = os.path.dirname(current_dir)
sys.path.append(path)

In [ ]:
import mintpy
import plotting_config

### Generate a toy dataset for testing 

In [ ]:
x_1 = 